# Weak Supervised Learning

## Importing Packages

In [23]:
import pandas as pd
import time
import json

import re
from itertools import chain
from collections import Counter

import numpy as np

In [3]:
RANDOM_STATE = 214

## Importing Data

In [34]:
df_pls_theme = pd.read_parquet('../../data/proposicoes_temas_one_hot_encoding.parquet')

Print basic information.

In [35]:
df_pls_theme.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65977 entries, 0 to 65976
Data columns (total 27 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   id                                           65977 non-null  int64 
 1   ementa                                       65977 non-null  object
 2   Administração Pública                        65977 non-null  int64 
 3   Agricultura, Pecuária, Pesca e Extrativismo  65977 non-null  int64 
 4   Arte, Cultura e Religião                     65977 non-null  int64 
 5   Cidades e Desenvolvimento Urbano             65977 non-null  int64 
 6   Comunicações                                 65977 non-null  int64 
 7   Defesa e Segurança                           65977 non-null  int64 
 8   Direito Civil e Processual Civil             65977 non-null  int64 
 9   Direito Penal e Processual Penal             65977 non-null  int64 
 10  Direito e 

## Data exploration

In [36]:
BINARY_CLASS = "Comunicações"
IN_BINARY_CLASS = "in_" + BINARY_CLASS.lower().replace(" ", "_")

df_pls_theme = df_pls_theme.drop_duplicates(subset=["ementa"])
df_pls_theme = df_pls_theme[["id", "ementa", BINARY_CLASS]]
df_pls_theme = df_pls_theme.rename(
    columns={BINARY_CLASS: IN_BINARY_CLASS}
)

In [37]:
df_pls_theme.query(f"{IN_BINARY_CLASS} == 1")

tema,id,ementa,in_comunicações
6,14925,Estabelece penalidades pelo uso de telefone ce...,1
12,14933,Dispõe sobre a transmissão da TV Câmara e da T...,1
33,14955,Dispõe sobre regulamentação de anúncios public...,1
37,14959,"Proíbe a utilização de recursos públicos, no â...",1
190,15143,Regulamenta o inciso III do art. 221 da Consti...,1
...,...,...,...
65866,2359938,"Altera a Lei nº 4.117, de 27 de agosto de 1962...",1
65870,2360003,"Altera as Leis nºs 9.998, de 17 de agosto de 2...",1
65874,2360115,"Proíbe a publicação, distribuição ou transmiss...",1
65959,2364284,Dispõe sobre a universalização da internet na ...,1


Sampling some examples to seek patterns and keywords.

In [38]:
df_pls_theme.query(f"{IN_BINARY_CLASS} == 1")['ementa'].tolist()[-360:-350]

['Altera a Lei Nº 12.965, de 23 de abril de 2014 (Marco Civil da Internet), e a Lei Nº 13.709, de 14 de agosto de 2018 (Lei Geral de Proteção de Dados), com a finalidade de estabelecer regras para a utilização de pseudônimos e perfis anônimos na Internet.',
 'Altera o art. 1.788 da Lei n.º 10.406, de 10 de janeiro de 2002.',
 '"Acrescenta o art. 10-A à","(Marco Civil da Internet), a fim de dispor sobre a destinação das contas de aplicações de internet após a morte de seu titular."',
 'Reduz as tarifas de frete para o envio, pela Empresa Brasileira de Correios e Telégrafos – CORREIOS, de objeto postal dos quais são remetentes as microempresas, as empresas de pequeno porte e microempresários individuais, nos termos do art. 34 da Lei nº 6.538, de 22 de junho de 1978, durante a vigência do estado de calamidade pública reconhecido pelo Decreto Legislativo nº 6, de 20 de março de 2020.',
 '"Estabelece a obrigatoriedade de diploma de Jornalismo em curso superior, no serviço público e dá outra

In [39]:
# jornais, revistas, rádio, televisão, mídias sociais, internet, blogs, sites, portais, aplicativos, emissoras, videocassetes, DVDs, CDs,
# telecomunicação, TV, emissoras, redes, redes de computadores
# telefone, telefonia, ligação, ligações, chamadas, chamada, chamadas telefônicas, chamada telefônica,
# veiculação, veiculação gratuita, 
# propaganda, publicidade, comunicações, comunicações, aparelhos
# rótulo, cartaz, outdoor, faixa, banner, panfleto, folheto, folder, volante, adesivo, adesivos,
# vídeo, streaming, celulares, 1G, 2G, 3G, 4G, 5G, web, mobile, dados, banda larga, digital, redes sociais
# pré-pago, pós-pago, telemarketing, sons, imagens, libras, 

# educação, escola, escolas, ensino, ensino fundamental, ensino médio, ensino superior, universidade, universidades, faculdade, faculdades,
# crianças, adolescentes, educação, educativo

# fake news, Marco Civil da Internet, Lei Geral de Proteção de Dados, LGPD, Lei de Acesso à Informação, LAI, Lei de Imprensa, Lei de Telecomunicações

Detecting the most frequently cited laws.

In [40]:
law_pattern = re.compile(r"lei nº \d{1,3}\.{0,1}\d{3}", re.IGNORECASE)

laws_list = df_pls_theme.query(f"{IN_BINARY_CLASS} == 1").ementa.apply( lambda x: re.findall(law_pattern, x) ).tolist()
laws_list = chain.from_iterable(laws_list)
laws_list = [law.lower() for law in laws_list]

# count laws
laws_counter = Counter(laws_list)
laws_counter.most_common(20)
# 9.472, 12.965, 4.117, 9.998, 12.485, 9.612, 8.078, 9.294
# 8.069
# 9.610

[('lei nº 9.472', 290),
 ('lei nº 12.965', 99),
 ('lei nº 4.117', 76),
 ('lei nº 9.998', 52),
 ('lei nº 12.485', 50),
 ('lei nº 9.612', 46),
 ('lei nº 8.078', 36),
 ('lei nº 9.294', 32),
 ('lei nº 2.848', 23),
 ('lei nº 8.069', 21),
 ('lei nº 9.610', 20),
 ('lei nº 9.504', 17),
 ('lei nº 8.977', 14),
 ('lei nº 6.538', 13),
 ('lei nº 5.250', 12),
 ('lei nº 5.070', 11),
 ('lei nº 10.098', 11),
 ('lei nº 12.527', 10),
 ('lei nº 10.703', 9),
 ('lei nº 12.232', 9)]

## Data splitting

Using the pre-defined labels to get a sense of the distribution.

In [41]:
df_size = len(df_pls_theme)
df_size_class = len(df_pls_theme.query(f"{IN_BINARY_CLASS} == 1"))

print(df_size)
print(df_size_class)
print(100*df_size_class / df_size)

61934
2254
3.6393580262860463


Sampling 5% of the data to create the test and validation sets.

In [42]:
n_samples = 2*int(np.ceil(df_size*0.5/100))

df_pls_theme_shuffle = df_pls_theme.sample(frac=1.0, random_state=RANDOM_STATE)

In [44]:
df_pls_theme_test_val = df_pls_theme_shuffle[:n_samples]
df_pls_theme_train = df_pls_theme_shuffle[n_samples:]


len(df_pls_theme_test_val), len(df_pls_theme_train)

(620, 61314)

Saving the datasets.

In [47]:
df_pls_theme_test_val.to_csv("./data/weak_supervised_learning_test_val.csv", index=False)
df_pls_theme_train.to_csv("./data/weak_supervised_learning_train.csv", index=False)

UnicodeEncodeError: 'latin-1' codec can't encode character '\u201d' in position 107: ordinal not in range(256)